## Demo notebook to detect whether people are using safety gear or not

The data is taken from <br>


imports ...

In [24]:
import os 
import torch
from torch import nn, optim
import torchvision
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
import torchvision.transforms.v2 as v2
import math
import time 

Parametes 

In [12]:
dataset_folder  = "../Dataset"
batch_size = 32
train_ratio = 0.85
valid_ratio = 0.0
test_ratio = 0.15
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

print (f"Settings ... \n dataset path \t\t {dataset_folder} \n batch_size \t\t {batch_size}" +
       f"\n train_ratio \t\t {train_ratio} \n validation ratio \t {valid_ratio} \n test ratio \t\t {test_ratio} \n device \t\t {device}")

Using device: cpu
Settings ... 
 dataset path 		 ../Dataset 
 batch_size 		 32
 train_ratio 		 0.85 
 validation ratio 	 0.0 
 test ratio 		 0.15 
 device 		 cpu


Check the classes in the folder

In [5]:
class_labels = [name for name in os.listdir(dataset_folder) if not name.startswith(".DS_Store")]
print(class_labels)
num_classes = len(class_labels)
print(num_classes)

['Wearing Safety Gear', 'Not Wearing Safety Gear']
2


Creating dataset from the image folder 

In [6]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

transformations = v2.Compose([
    v2.ToImage(),
    v2.RandomHorizontalFlip(),
    v2.RandomVerticalFlip(),
    v2.RandomResizedCrop(size=(256, 256), antialias=True),
    v2.RandomRotation(degrees = (0, 170)),
    v2.ToDtype(dtype = torch.float32, scale = True),
    v2.Resize(size = (256, 256), antialias = True),
    v2.Normalize(mean, std)
])

dataset = ImageFolder(dataset_folder, transform = transformations)


probe stats from the Image folder 

In [10]:
print (f"classes {dataset.classes}")
d = dataset.class_to_idx

classes ['Not Wearing Safety Gear', 'Wearing Safety Gear']


splitting the dataset <br>
this is a small dataset, we ar eonly using train and test dataset 

In [17]:
train_size = math.ceil(len(dataset) * train_ratio)
test_szie = math.floor(len(dataset) * test_ratio)
print(f"dataset {len(dataset)}, train dataset {train_size}, test size {test_szie}")

train_dataset, test_dataset = random_split(dataset,[train_size, test_szie])

train_dataloader = DataLoader(train_dataset,shuffle=True,batch_size=batch_size)
test_dataloader = DataLoader(test_dataset,shuffle=True, batch_size=batch_size)

print(f"Lenght of training dataset {len(train_dataloader.dataset)}")
print(f"Lenght of test dataset {len(test_dataloader.dataset)}")

dataset 355, train dataset 302, test size 53
Lenght of training dataset 302
Lenght of test dataset 53


try ResNet 

In [22]:
res_net = torchvision.models.resnet50(weights='IMAGENET1K_V1')

for param in res_net.parameters():
    param.requires_grad = False

res_net.fc  = torch.nn.Sequential(
    nn.Linear(2048,128),
    nn.ReLU(),
    nn.Linear(
        in_features=128,
        out_features=num_classes
    ),
)



ANy other model(s) to evalaute can go here ... 

In [23]:
model_to_evaluate = res_net

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(res_net.fc.parameters())

training method 

In [ ]:
def train_model(model,criterion,optimier, data_loader,device, num_epochs=0):
    model.to(device)
    for epoch in range(num_epochs):
        loss_batches = 0 
        corrects_batches = 0
        start_time = time.time()
        for x,y in data_loader:
            
            x,y = x.to(device), y.to(device)
            
            outputs = model(x)
            loss = criterion(outputs,y)
            optimier.zero_grad()
            loss.backward()
            optimier.step()
            _,preds = torch.max(outputs,1)
            loss_batches += loss.item() * x.size(0)
            corrects_batches += torch.sum(preds == y.data)
        epoch_loss = loss_batches / len(data_loader)
        epoch_acc = corrects_batches.double() / len(data_loader)
        print(f"\n epoch {epoch} Loss : {epoch_loss:.4f} Accuracy {epoch_acc:.2f} time {(time.time()-start_time):.03d} secs")

    return model 
